In [1]:
from gcms_spectra_gnn.spectra_dataset import MoleculesDataset
from tqdm.notebook import tqdm
import json

In [2]:
mgf_file = 'data/NIST_GC_EI_MAX_LIBRARY.mgf'
tsv_file = 'data/NIST_GC_EI_MAX_v2.tsv'
mgf_json_file = 'data/NIST_GC_EI_MAX_LIBRARY.json'

In [ ]:
with open(mgf_file) as fh:
    library_lines = fh.read().splitlines()

def parse_entry(entry):
    separated = [value.split('=', 1) for value in entry]
    i = 0
    key_value = separated[i]
    entries = dict()
    m_z, intensity = [], []
    while len(key_value) == 2:
        key, value = key_value
        entries[key] = value
        i += 1
        key_value = separated[i]
    while i < len(separated):
        key_value = separated[i]
        key, value = key_value[0].split(' ')
        m_z.append(float(key))
        intensity.append(float(value))
        i += 1
    entries['M/Z'] = m_z
    entries['INTENSITY'] = intensity
    return entries

ENTRY_START = 'BEGIN IONS'
ENTRY_END = 'END IONS'

library_entries = []
current_entry = []
i = 0
pbar = tqdm(total=len(library_lines))
while i < len(library_lines):
    line = library_lines[i]
    if line == ENTRY_START:
        current_entry = []
    elif line == ENTRY_END:
        library_entries.append(parse_entry(current_entry))
        current_entry = []
    else:
        current_entry.append(line)
        
    pbar.update()
    i += 1

with open(mgf_json_file, 'w') as fh:
    json.dump(library_entries, fh)